This file more or less describes the timeline of problems that we experienced in our case study attempts. 


The first cell block imports all packages and files that are required through all attempts. 

In [1]:
import sys
sys.path.append("../..")
import numpy as np
import pandas as pd
import geopandas as gpd
from dyntapy.assignments import StaticAssignment
from dyntapy.toll import create_toll_object
from dyntapy.demand_data import od_graph_from_matrix
from pickle import load 
import warnings
warnings.filterwarnings('ignore') # hide warnings

network_path = "C:/Users/anton/IP2/toll_optimization/data_map/HEEDS_input/network_with_centroids/elastic_BRUSSEL_40"
od_path = "C:/Users/anton/IP2/toll_optimization/data_map/HEEDS_input/od_graph/elastic_BRUSSEL_40.xlsx"
A_matrix_path = "C:/Users/anton/IP2/toll_optimization/data_map/HEEDS_input/elastic/A_matrix_BRUSSEL_40"
B_matrix_path = "C:/Users/anton/IP2/toll_optimization/data_map/HEEDS_input/elastic/B_matrix_BRUSSEL_40"
x_centroids_path = "C:/Users/anton/IP2/toll_optimization/data_map/HEEDS_input/elastic/x_centroids_BRUSSEL_40"
y_centroids_path = "C:/Users/anton/IP2/toll_optimization/data_map/HEEDS_input/elastic/y_centroids_BRUSSEL_40"

with open(network_path, 'rb') as network_file:
    g = load(network_file)
with open(A_matrix_path, 'rb') as matrix:
    A = load(matrix)
with open(B_matrix_path, 'rb') as matrix:
    B = load(matrix)
old_od = pd.read_excel(od_path)
old_od = old_od.to_numpy()
x_centroids = np.loadtxt(x_centroids_path)
y_centroids = np.loadtxt(y_centroids_path)
original_od_graph = od_graph_from_matrix(old_od,x_centroids,y_centroids) 

-----
    FIRST ATTEMPT: TOLLING BASED ON OUR DEFINED CORDON OUTSIDE RING OF BRUSSELS

TODO: figuur opslaan van dit cordon voor referentie (idealiter staat die figuur ook in verslag dus gwn ergens toevoegen hier)

In [2]:
toll_method = 'cordon'
toll_value = 0.05

# Load links for cordon outside ring of Brussels. 
links_path = "C:/Users/anton/IP2/toll_optimization/data_map/QGIS/ring_cordon/links_crossing_cordon.shp"
links_crossing_cordon = gpd.read_file(links_path)
toll_link_ids = [toll_id for toll_id in links_crossing_cordon['link_id']]

# Test STA on our cordon tolling scheme fails... 
toll_object = create_toll_object(g, toll_method, toll_link_ids, toll_value)
assignment = StaticAssignment(g, original_od_graph, toll_object)
result = assignment.run('dial_b')

init passed successfully
initial loading starts 


AssertionError: 

Opmerking voor structuur: we wisten van toy case dat een zone of cordon van 2 links gewoon werkt naar behoren
We wisten ook dat een single toll prima werkt op brussels, echter cordon van 150 links ofzo werkt niet
Vraag is dan waar de overgang is van werken naar niet werken, daarom dat we onze toll links verkleinen. 

-----
    SECOND ATTEMPT: TOLLING BASED ON THE DEFINED PENTAGON CORDON

TODO: figuur opslaan van dit cordon voor referentie (idealiter staat die figuur ook in verslag dus gwn ergens toevoegen hier)

In [3]:
toll_method = 'cordon'
toll_value = 0.05

# Load links for pentagon defined cordon. 
links_path = "C:/Users/anton/IP2/toll_optimization/data_map/QGIS/pentagon/links_crossing_pentagon.shp"
links_crossing_cordon = gpd.read_file(links_path)
toll_link_ids = [toll_id for toll_id in links_crossing_cordon['link_id']]

# Test STA on this tolling scheme also fails... 
toll_object = create_toll_object(g, toll_method, toll_link_ids, toll_value)
assignment = StaticAssignment(g, original_od_graph, toll_object)
result = assignment.run('dial_b')

init passed successfully
initial loading starts 


AssertionError: 

Go even smaller, besproken met gaurav om manueel wat links te proberen

------
    THIRD ATTEMPT: TOLLING THE E19 AND E40 JUNCTIONS TO THE RING OF BRUSSELS

TODO: figuur van dit geval (idealiter staat die figuur ook in verslag dus gwn ergens toevoegen hier)

In [4]:
toll_method = 'cordon'
toll_value = 0.05
toll_link_ids = [642, 654, 760, 939, 1347, 1354, 1367, 2970, 6355, 6724, 10692, 10693, 15600, 15845, 15848]

# Test STA on this tolling scheme also fails... 
toll_object = create_toll_object(g, toll_method, toll_link_ids, toll_value)
assignment = StaticAssignment(g, original_od_graph, toll_object)
result = assignment.run('dial_b')

init passed successfully
initial loading starts 


AssertionError: 

Nog steeds zelfde error. nog minder links

-----
    FOURTH ATTEMPT: TOLLING THE E19 JUNCTION TO THE RING OF BRUSSELS

TODO: figuur opslaan van dit cordon voor referentie (idealiter staat die figuur ook in verslag dus gwn ergens toevoegen hier)

In [5]:
toll_method = 'cordon'
toll_value = 0.05
toll_link_ids = [939, 1347, 2970, 6355, 6724, 15600]

# Test STA on this tolling scheme also fails... 
toll_object = create_toll_object(g, toll_method, toll_link_ids, toll_value)
assignment = StaticAssignment(g, original_od_graph, toll_object)
result = assignment.run('dial_b')

To cover: 
- STA zonder toll
- STA met toll
- network check
- vergelijken flow en cost + waar wordt de toll op gezet (van links)

print(np.max(abs(new_od)))

print(np.max(abs(old_od)))

print(np.max(abs((new_od - old_od))))

print(abs((np.linalg.norm(new_od) - np.linalg.norm(old_od))/np.linalg.norm(old_od)))

In [ ]:
# TODO: both for regular STA as STA with toll needed here. 
flow_results = result.flows
for id in toll_link_ids:
    print(id, "_", flow_results[id])

cost_results = result.link_costs
for id in toll_link_ids:
    print(id, "_", cost_results[id])

t = 0
for id in range(len(cost_results)):
    if cost_results[id] > 0.1:
        print(id)
        t += 1
print(t)
